# 图和会话

Tensorflow使用数据力图将计算表示独立指令之间的依赖关系。这可生成低级别的编程模型，该模型中，您首先定义数据流图，然后创建tensorflow会话，以便在一组本地和远程设备上运行图的各个部分。

## 为什么使用数据流图？

数据流是一种并行计算常用的编程模型，载数据流图中，节点表示计算单元，边缘表示计算使用或产生的数据。

在执行您的程序时，数据流可以为 TensorFlow 提供多项优势：

1）并行处理。 通过使用明确的边缘来表示操作之间的依赖关系，系统可以轻松识别能够并行执行的操作。

2）分布式执行。 通过使用明确的边缘来表示操作之间流动的值，TensorFlow 可以将您的程序划分到连接至不同机器的多台设备上（CPU、GPU 和 TPU）。TensorFlow 将在这些设备之间进行必要的通信和协调。

3）编译。 TensorFlow 的 XLA 编译器可以使用数据流图中的信息生成更快的代码，例如将相邻的操作融合到一起。

4）可移植性。 数据流图是一种不依赖于语言的模型代码表示法。您可以使用 Python 构建数据流图，将其存储在 SavedModel 中，并使用 C++ 程序进行恢复，从而实现低延迟的推理。

## 什么是 tf.Graph？

tf.Graph 包含两类相关信息：

图结构。 图的节点和边缘，表示各个操作组合在一起的方式，但不规定它们的使用方式。图结构与汇编代码类似：检查图结构可以传达一些有用的信息，但它不包含源代码传达的所有实用上下文信息。

图集合。 TensorFlow 提供了一种在 tf.Graph 中存储元数据集合的通用机制。tf.add_to_collection 函数允许您将对象列表与一个键关联（其中 tf.GraphKeys 定义了部分标准键），tf.get_collection 允许您查询与某个键关联的所有对象。TensorFlow 库的许多部分会使用此设施资源：例如，当您创建 tf.Variable 时，系统会默认将其添加到表示“全局变量”和“可训练变量”的集合中。当您后续创建 tf.train.Saver 或 tf.train.Optimizer 时，这些集合中的变量将用作默认参数

## 构建tf.Graph

大多数 TensorFlow 程序都以数据流图构建阶段开始。在此阶段，您会调用 TensorFlow API 函数，这些函数可构建新的 tf.Operation（节点）和 tf.Tensor（边缘）对象并将它们添加到 tf.Graph 实例中。TensorFlow 提供了一个默认图，此图是同一上下文中的所有 API 函数的明确参数。例如：

调用 tf.constant(42.0) 可创建单个 tf.Operation，该操作可以生成值 42.0，将该值添加到默认图中，并返回表示常量值的 tf.Tensor。

调用 tf.matmul(x, y) 可创建单个 tf.Operation，该操作会将 tf.Tensor 对象 x 和 y 的值相乘，将其添加到默认图中，并返回表示乘法运算结果的 tf.Tensor。

执行 v = tf.Variable(0) 可向图添加一个 tf.Operation，该操作可以存储一个可写入的张量值，该值在多个 tf.Session.run 调用之间保持恒定。tf.Variable 对象会封装此操作，并可以像张量一样使用，即读取已存储的值的当前值。tf.Variable 对象也具有 assign 和 assign_add 等方法，这些方法可创建 tf.Operation 对象，这些对象在执行时将更新已存储的值。（请参阅变量了解关于变量的更多信息。）

调用 tf.train.Optimizer.minimize 可将操作和张量添加到计算梯度的默认图中，并返回一个 tf.Operation，该操作在运行时会将这些梯度应用到一组变量上。

## 命名指令

tf.Graph 对象会为其包含的 tf.Operation 对象定义一个命名空间。TensorFlow 会自动为您的图中的每个指令选择一个唯一名称，但您也可以指定描述性名称，使您的程序阅读和调试起来更轻松。TensorFlow API 提供两种方法来覆盖操作名称：

如果 API 函数会创建新的 tf.Operation 或返回新的 tf.Tensor，则会接受可选 name 参数。例如，tf.constant(42.0, name="answer") 会创建一个新的 tf.Operation（名为 "answer"）并返回一个 tf.Tensor（名为 "answer:0"）。如果默认图已包含名为 "answer" 的操作，则 TensorFlow 会在名称上附加 "_1"、"_2" 等字符，以便让名称具有唯一性。

借助 tf.name_scope 函数，您可以向在特定上下文中创建的所有操作添加名称作用域前缀。当前名称作用域前缀是一个用 "/" 分隔的名称列表，其中包含所有活动 tf.name_scope 上下文管理器的名称。如果某个名称作用域已在当前上下文中被占用，TensorFlow 将在该作用域上附加 "_1"、"_2" 等字符。例如：

In [7]:
c_0 = tf.constant(0, name="c")  # => operation named "c"

# Already-used names will be "uniquified".
c_1 = tf.constant(2, name="c")  # => operation named "c_1"

# Name scopes add a prefix to all operations created in the same context.
with tf.name_scope("outer"):
  c_2 = tf.constant(2, name="c")  # => operation named "outer/c"

  # Name scopes nest like paths in a hierarchical file system.
  with tf.name_scope("inner"):
    c_3 = tf.constant(3, name="c")  # => operation named "outer/inner/c"

  # Exiting a name scope context will return to the previous prefix.
  c_4 = tf.constant(4, name="c")  # => operation named "outer/c_1"

  # Already-used name scopes will be "uniquified".
  with tf.name_scope("inner"):
    c_5 = tf.constant(5, name="c")  # => operation named "outer/inner_1/c"

print(c_0)
print(c_1)
print(c_2)
print(c_3)
print(c_4)
print(c_5)

Tensor("c_8:0", shape=(), dtype=int32)
Tensor("c_9:0", shape=(), dtype=int32)
Tensor("outer_4/c:0", shape=(), dtype=int32)
Tensor("outer_4/inner/c:0", shape=(), dtype=int32)
Tensor("outer_4/c_1:0", shape=(), dtype=int32)
Tensor("outer_4/inner_1/c:0", shape=(), dtype=int32)


## 将操作放置到不同的设备上

如果您希望 TensorFlow 程序使用多台不同的设备，则可以使用 tf.device 函数轻松地请求将在特定上下文中创建的所有操作放置到同一设备（或同一类型的设备）上。

设备规范具有以下形式：

/job:<JOB_NAME>/task:<TASK_INDEX>/device:<DEVICE_TYPE>:<DEVICE_INDEX>
其中：

<JOB_NAME> 是一个字母数字字符串，并且不以数字开头。
<DEVICE_TYPE> 是一种注册设备类型（例如 GPU 或 CPU）。
<TASK_INDEX> 是一个非负整数，表示名为 <JOB_NAME> 的作业中的任务的索引。请参阅 tf.train.ClusterSpec 了解作业和任务的说明。
<DEVICE_INDEX> 是一个非负整数，表示设备索引，例如用于区分同一进程中使用的不同 GPU 设备。
您无需指定设备规范的每个部分。例如，如果您在具有单个 GPU 的单机器配置中运行，您可以使用 tf.device 将一些操作固定到 CPU 和 GPU 上：

如果您在典型的分布式配置中部署 TensorFlow，您可以指定作业名称和任务 ID，以便将变量放到参数服务器作业 ("/job:ps") 中的任务上，并将其他操作放置到工作器作业 ("/job:worker") 中的任务上：

借助 tf.device，您可以高度灵活地选择单个操作或 TensorFlow 图各个区域的放置方式。在很多情况下，简单的启发法具有良好的效果。例如，tf.train.replica_device_setter API 可与 tf.device 结合使用，以针对数据并行分布式训练放置操作。例如，以下代码段展示了 tf.train.replica_device_setter 如何将不同的放置政策应用于 tf.Variable 对象和其他操作：

## 类似于张量的对象

类张量对象包括以下类型的元素：

tf.Tensor

tf.Variable

numpy.ndarray

list（以及类似于张量的对象的列表）

标量 Python 类型：bool、float、int、str

## 在以下会话中执行图：tf.Session

tf.Session 对象使我们能够访问本地机器中的设备和使用分布式 TensorFlow 运行时的远程设备。它还可缓存关于 tf.Graph 的信息，使您能够多次高效地运行同一计算。

### 创建 tf.Session

### 使用 tf.Session.run 执行操作

tf.Session.run 方法是运行 tf.Operation 或评估 tf.Tensor 的主要机制。您可以将一个或多个 tf.Operation 或 tf.Tensor 对象传递到 tf.Session.run，TensorFlow 将执行计算结果所需的操作。

In [9]:
x = tf.constant([[37.0, -23.0], [1.0, 4.0]])
w = tf.Variable(tf.random_uniform([2, 2]))
y = tf.matmul(x, w)
output = tf.nn.softmax(y)
init_op = w.initializer

with tf.Session() as sess:
  # Run the initializer on `w`.
  sess.run(init_op)

  # Evaluate `output`. `sess.run(output)` will return a NumPy array containing
  # the result of the computation.
  print(sess.run(output))

  # Evaluate `y` and `output`. Note that `y` will only be computed once, and its
  # result used both to return `y_val` and as an input to the `tf.nn.softmax()`
  # op. Both `y_val` and `output_val` will be NumPy arrays.
  y_val, output_val = sess.run([y, output])

[[0.01772438 0.9822756 ]
 [0.724863   0.27513704]]


 tf.Session.run 也可以选择接受 Feed 字典，该字典是从 tf.Tensor 对象（通常是 tf.placeholder 张量）到在执行时会被替换为这些张量的值（通常是 Python 标量、列表或 NumPy 数组）的映射。

In [12]:
# Define a placeholder that expects a vector of three floating-point values,
# and a computation that depends on it.
x = tf.placeholder(tf.float32, shape=[3])
y = tf.square(x)

with tf.Session() as sess:
  # Feeding a value changes the result that is returned when you evaluate `y`.
  print(sess.run(y, {x: [1.0, 2.0, 3.0]}))  # => "[1.0, 4.0, 9.0]"
  print(sess.run(y, {x: [0.0, 0.0, 5.0]}))  # => "[0.0, 0.0, 25.0]"

  # Raises <a href="../api_docs/python/tf/errors/InvalidArgumentError"><code>tf.errors.InvalidArgumentError</code></a>, because you must feed a value for
  # a `tf.placeholder()` when evaluating a tensor that depends on it.
  #sess.run(y)

  # Raises `ValueError`, because the shape of `37.0` does not match the shape
  # of placeholder `x`.
 # sess.run(y, {x: 37.0})

[1. 4. 9.]
[ 0.  0. 25.]


tf.Session.run 也接受可选的 options 参数（允许您指定与调用有关的选项）和可选的 run_metadata 参数（允许您收集与执行有关的元数据）。例如，您可以同时使用这些选项来收集与执行有关的跟踪信息：

## 直观展示您的图

TensorFlow 包含可帮助您理解图中的代码的工具。图可视化工具是 TensorBoard 的一个组件，可在浏览器中可视化图的结构。要创建可视化图表，最简单的方法是传递 tf.Graph（在创建 tf.summary.FileWriter 时）：

## 使用多个图进行编程

如上所述，TensorFlow 提供了一个“默认图”，此图明确传递给同一上下文中的所有 API 函数。对于许多应用而言，单个图便已足够。但是，TensorFlow 还提供了操作默认图的方法，在更高级的用例中，这些方法可能十分有用。例如：

tf.Graph 会定义 tf.Operation 对象的命名空间：单个图中的每个操作必须具有唯一名称。如果请求的名称已被占用，TensorFlow 将在操作名称上附加 "_1"、"_2" 等字符，以便确保名称的唯一性。通过使用多个明确创建的图，您可以更有效地控制为每个操作指定什么样的名称。

默认图会存储与添加的每个 tf.Operation 和 tf.Tensor 有关的信息。如果您的程序创建了大量未连接的子图，更有效的做法是使用另一个 tf.Graph 构建每个子图，以便回收不相关的状态。

您可以安装另一个 tf.Graph 作为默认图（使用 tf.Graph.as_default 上下文管理器）：

In [15]:
g_1 = tf.Graph()
with g_1.as_default():
  # Operations created in this scope will be added to `g_1`.
  c = tf.constant("Node in g_1")

  # Sessions created in this scope will run operations from `g_1`.
  sess_1 = tf.Session()

g_2 = tf.Graph()
with g_2.as_default():
  # Operations created in this scope will be added to `g_2`.
  d = tf.constant("Node in g_2")

# Alternatively, you can pass a graph when constructing a <a href="../api_docs/python/tf/Session"><code>tf.Session</code></a>:
# `sess_2` will run operations from `g_2`.
sess_2 = tf.Session(graph=g_2)

assert c.graph is g_1
assert sess_1.graph is g_1

assert d.graph is g_2
assert sess_2.graph is g_2

In [17]:
# Print all of the operations in the default graph.
g = tf.get_default_graph()
print(g.get_operations())

[<tf.Operation 'c' type=Const>, <tf.Operation 'c_1' type=Const>, <tf.Operation 'outer/c' type=Const>, <tf.Operation 'outer/inner/c' type=Const>, <tf.Operation 'outer/c_1' type=Const>, <tf.Operation 'outer/inner_1/c' type=Const>, <tf.Operation 'c_2' type=Const>, <tf.Operation 'c_3' type=Const>, <tf.Operation 'outer_1/c' type=Const>, <tf.Operation 'outer_1/inner/c' type=Const>, <tf.Operation 'outer_1/c_1' type=Const>, <tf.Operation 'outer_1/inner_1/c' type=Const>, <tf.Operation 'c_4' type=Const>, <tf.Operation 'c_5' type=Const>, <tf.Operation 'outer_2/c' type=Const>, <tf.Operation 'outer_2/inner/c' type=Const>, <tf.Operation 'outer_2/c_1' type=Const>, <tf.Operation 'outer_2/inner_1/c' type=Const>, <tf.Operation 'c_6' type=Const>, <tf.Operation 'c_7' type=Const>, <tf.Operation 'outer_3/c' type=Const>, <tf.Operation 'outer_3/inner/c' type=Const>, <tf.Operation 'outer_3/c_1' type=Const>, <tf.Operation 'outer_3/inner_1/c' type=Const>, <tf.Operation 'c_8' type=Const>, <tf.Operation 'c_9' type